MINIPROJECT : HATE SPEECH DETECTION

In [4]:
#import libraries
import pandas as pd
import numpy as np
import re 
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = nltk.SnowballStemmer("english")
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_hate=pd.read_csv("dataset.csv")

In [6]:
df_hate.shape

(24783, 7)

In [7]:
df_hate.head(10) #reading the dataset

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [8]:
df_hate.info() #checking the info of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [9]:
df_hate.isnull().sum() #to check if dataset contains null values

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [10]:
hate_set=df_hate.drop(['Unnamed: 0'],axis=1)

In [11]:
hate_set.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [12]:
hate_set["labels"]=hate_set["class"].map({0: "Hate speech",1: "Offensive language", 2: "no hate & offensive text"})

In [13]:
hate_set.head(10)

,count,hate_speech,offensive_language,neither,class,tweet,labels
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,no hate & offensive text
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive language
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive language
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive language
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive language
5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",Offensive language
6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",Offensive language
7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,Offensive language
8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",Offensive language
9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",Offensive language


In [14]:
hate_set=hate_set[["tweet","labels"]]

In [15]:
hate_set.head(10)

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,no hate & offensive text
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive language
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",Offensive language
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",Offensive language
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,Offensive language
8,""" &amp; you might not get ya bitch back &amp; ...",Offensive language
9,""" @rhythmixx_ :hobbies include: fighting Maria...",Offensive language


In [16]:
stopword=set(stopwords.words('english'))

def clean_special(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]', '', text)
    text=re.sub('https?://S+|www\.\S+', '', text)
    text=re.sub('<.*?>+', '', text)
    text=re.sub('[%s]' % re.escape(string.punctuation), '',text)
    text=re.sub('\n', '', text)
    text=re.sub('\w*\d\w*', '', text)
    text=[word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text=[stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
    



In [17]:
hate_set["tweet"]=hate_set["tweet"].apply(clean_special)
hate_set.head(10)

,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,no hate & offensive text
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,Offensive language
2,rt urkindofbrand dawg rt ever fuck bitch sta...,Offensive language
3,rt cganderson vivabas look like tranni,Offensive language
4,rt shenikarobert shit hear might true might f...,Offensive language
5,tmadisonx shit blow meclaim faith somebodi sti...,Offensive language
6,brighterday sit hate anoth bitch got much shi...,Offensive language
7,caus im tire big bitch come us skinni,Offensive language
8,amp might get ya bitch back amp that,Offensive language
9,rhythmixx hobbi includ fight mariambitch,Offensive language


In [18]:
x=np.array(hate_set["tweet"])
y=np.array(hate_set["labels"])

In [19]:
#converting text into numerical values
v=CountVectorizer()
X=v.fit_transform(x) #fit the data

In [20]:
#dividing the dataset into training and testing set
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

best model to predict hate speech

In [18]:
#naive Bayes
# model_nb=MultinomialNB()
# model_nb.fit(X_train,y_train)
# model_nb.score(X_test,y_test)

In [19]:
# prep_nb=model_nb.predict(X_test)
# cm=confusion_matrix(y_test,prep_nb)
# print(cm)

In [20]:
# random forest
# model_rf=RandomForestClassifier()
# model_rf.fit(X_train,y_train)
# model_rf.score(X_test,y_test)

In [21]:
# prep_rf=model_rf.predict(X_test)
# cm=confusion_matrix(y_test,prep_rf)

In [22]:
# print(cm)

In [21]:
from sklearn import tree

In [24]:
# support vector machine
# model_svm=svm.SVC()
# model_svm.fit(X_train,y_train)
# model_svm.score(X_test,y_test)

In [22]:
#decisionTreeclassifier
model_dc=tree.DecisionTreeClassifier()
model_dc.fit(X_train,y_train)
model_dc.score(X_test,y_test)

0.8664873456412764

In [24]:
user=input("input text: ")
hate_set=v.transform([user]).toarray()
output=model_dc.predict(hate_set)
print(output)


input text: lets kill people
['Hate speech']


In [25]:
import pickle
with open('model.pickle','wb') as f:
    pickle.dump(model_dc,f)

In [26]:
vec_file='vectorizer.pickle'
pickle.dump(v,open(vec_file,'wb'))